# <center>Sort

## 56. Merge Intervals (medium)

Given a collection of intervals, merge all overlapping intervals.

https://leetcode-cn.com/problems/merge-intervals/

In [3]:
public int[][] merge(int[][] intervals) {
    // 首先对区间数组按起点从小到大进行排序，这样合并操作可以从左至右依序进行
    Arrays.sort(intervals, new Comparator<int[]>(){
        public int compare(int[] arr1, int[] arr2){
            return arr1[0] - arr2[0];
        }
    });
    
    // 思路一：用一个指针i遍历区间，若i+1的起点小于i的终点，就进行合并，直到i+1的起点大于i的终点
    List<int[]> res = new ArrayList<>();
    int i = 0;
    while(i < intervals.length){
        int left = intervals[i][0];   // 第i个区间的起点
        int right = intervals[i][1];  // 第i个区间的终点
        while(i < intervals.length - 1 && right >= intervals[i + 1][0]){ // 因为要取i+1，所以i < length-1
            i++;
            right = Math.max(right, intervals[i][1]);  // 更新终点，注意取最大值
        }
        // 直到i的终点(right)小于i+1的起点，将left到right合并
        res.add(new int[]{left, right});
        i++; //继续看下一个区间
    }
    return res.toArray(new int[res.size()][]);
}


public int[][] merge2(int[][] intervals) {
    // 首先对区间数组按起点从小到大进行排序，这样合并操作可以从左至右依序进行
    Arrays.sort(intervals, new Comparator<int[]>(){
        public int compare(int[] arr1, int[] arr2){
            return arr1[0] - arr2[0];
        }
    });
    
    // 思路二：对于当前区间，依次看它后面的区间能否合并
    List<int[]> res = new ArrayList<>();
    Set<Integer> merged = new HashSet<>(); // 用来保存已合并过的区间，合并过的区间不再重复访问
    for(int i = 0; i < intervals.length; i++){
        if(merged.contains(i)) continue;
        int[] cur = intervals[i];
        for(int j = i + 1; j < intervals.length; j++){ //因为已排序，所以从i+1开始看
            if(intervals[j][0] >= cur[1]){
                cur[1] = Math.max(cur[1], intervals[j][1]);
                merged.add(j);
            } else break;
        }
        res.add(cur);
    }
    return res.toArray(new int[res.size()][]);
}